In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir('..')
import joblib
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
# feature columns
model_dir = "./models_1578_csv"
# feature columns
X_cols = [
    "hour","dayofweek","is_weekend","month",
    # "lag_24","rolling_24",
    # "electricity_current",
    "airTemperature", "dewTemperature", "windSpeed",
    # "temp_lag_1h","dewTemperature_lag_1h", "windSpeed_lag_1h",
    # "sqft", 
    "sqm", "primaryspaceusage", "site_id", "building_id",
    "Chilledwater", "Hotwater"
]

In [3]:
# =============================
# MultiColumnLabelEncoder
# =============================
class MultiColumnLabelEncoder:
    def __init__(self, categorical_cols, unknown_token="unknown"):
        self.categorical_cols = categorical_cols
        self.unknown_token = unknown_token
        self.encoders = {}

    def fit(self, df: pd.DataFrame):
        for col in self.categorical_cols:
            le = LabelEncoder()
            values = df[col].fillna(self.unknown_token).astype(str)
            classes = values.unique().tolist()
            if self.unknown_token not in classes:
                classes.append(self.unknown_token)
            le.fit(classes)
            self.encoders[col] = le
        return self

    def transform(self, df: pd.DataFrame):
        df = df.copy()
        for col, le in self.encoders.items():
            values = df[col].fillna(self.unknown_token).astype(str)
            # Những giá trị không nằm trong classes_ được map sang unknown_token
            values = values.where(values.isin(le.classes_), self.unknown_token)
            df[col] = le.transform(values)
        return df

    def inverse_transform(self, df: pd.DataFrame):
        """Decode số đã encode về giá trị gốc"""
        df = df.copy()
        for col, le in self.encoders.items():
            if col in df.columns:
                df[col] = le.inverse_transform(df[col].astype(int))
        return df

    def fit_transform(self, df: pd.DataFrame):
        self.fit(df)
        return self.transform(df)

    def save(self, path: str):
        joblib.dump({
            "categorical_cols": self.categorical_cols,
            "unknown_token": self.unknown_token,
            "encoders": self.encoders
        }, path)

    @classmethod
    def load(cls, path: str):
        data = joblib.load(path)
        obj = cls(
            categorical_cols=data["categorical_cols"],
            unknown_token=data["unknown_token"]
        )
        obj.encoders = data["encoders"]
        return obj

# Forecast & đánh giá

In [4]:
forecast_horizon = 24
models = []

for h in range(forecast_horizon):
    model_path = os.path.join(model_dir, f"model_hour_{h+1}.pkl")
    if os.path.exists(model_path):
        model = joblib.load(model_path)
        models.append(model)
        print(f"Loaded model_hour_{h+1}.pkl")
    else:
        raise FileNotFoundError(f"{model_path} not found!")

Loaded model_hour_1.pkl
Loaded model_hour_2.pkl
Loaded model_hour_3.pkl
Loaded model_hour_4.pkl
Loaded model_hour_5.pkl
Loaded model_hour_6.pkl
Loaded model_hour_7.pkl
Loaded model_hour_8.pkl
Loaded model_hour_9.pkl
Loaded model_hour_10.pkl
Loaded model_hour_11.pkl
Loaded model_hour_12.pkl
Loaded model_hour_13.pkl
Loaded model_hour_14.pkl
Loaded model_hour_15.pkl
Loaded model_hour_16.pkl
Loaded model_hour_17.pkl
Loaded model_hour_18.pkl
Loaded model_hour_19.pkl
Loaded model_hour_20.pkl
Loaded model_hour_21.pkl
Loaded model_hour_22.pkl
Loaded model_hour_23.pkl
Loaded model_hour_24.pkl


In [5]:
def load_data(path):
    data_df = pd.read_csv(path)
    return data_df

In [6]:
from sklearn.inspection import permutation_importance

# lấy 1 model (ví dụ horizon = 1)
model = models[0]
data_encoded = "data_1578_csv/test_encode.csv"
data_frame = load_data(data_encoded)
# lấy 1 tập test nhỏ để test nhanh
# df_test = data_frame.sample(20000, random_state=42)
df_test = data_frame.copy()
X_test = df_test[X_cols]
y_test = df_test["Electricity"]

r = permutation_importance(
    model,
    X_test,
    y_test,
    n_repeats=5,
    random_state=42,
    scoring="neg_mean_absolute_error"
)

importances = pd.DataFrame({
    "feature": X_cols,
    "importance": r.importances_mean
}).sort_values("importance", ascending=False)

print(importances)


              feature  importance
7                 sqm   99.056515
10        building_id   42.339606
11       Chilledwater   33.508483
8   primaryspaceusage   29.181334
12           Hotwater   17.540473
0                hour    8.116297
9             site_id    5.088810
1           dayofweek    1.735711
5      dewTemperature    0.905242
4      airTemperature    0.698417
3               month    0.335651
2          is_weekend    0.182269
6           windSpeed    0.091959


In [7]:
data_encoded = "data_1578_csv/test_encode.csv"
data_frame = load_data(data_encoded)
data_frame["timestamp"] = pd.to_datetime(data_frame["timestamp"])
print(len(data_frame))

250965


In [8]:
data_frame["building_id"].unique()

array([73, 22, 86, 79, 13, 24,  0, 21,  3, 30,  6, 12, 54, 63, 16, 85, 49,
       56, 11, 50, 10, 23, 62, 60, 43, 17, 34, 84, 25,  4, 38, 47, 74, 72,
       65, 78, 76, 82, 14, 53, 68, 45, 61, 31, 59, 41, 58, 26, 39, 33, 83,
        9, 80, 27, 36, 44, 77,  5, 46, 67,  2, 64, 37,  1, 66, 75, 19, 81,
       20, 15, 40, 55, 70, 69, 48, 71, 18, 35, 28, 42,  8, 52, 51, 29, 57,
        7, 32])

In [9]:
# CACH 2
from datetime import timedelta
import os
import pandas as pd
import numpy as np

results_file = "models_1578_csv/df_result_test.csv"
metric_file = "models_1578_csv/metrics_by_building_with_avg.csv"
forecast_horizon = 1

# =============================
# 1. Load kết quả cũ (nếu có)
# =============================
if os.path.exists(results_file):
    df_existing = pd.read_csv(results_file, parse_dates=["timestamp", "t0"])
    existing_keys = set(
        zip(df_existing["t0"], df_existing["building_id"], df_existing["horizon"])
    )
else:
    existing_keys = set()

all_results = []

# =============================
# 2. Predict theo từng building
# =============================
for building_id, df_b in data_frame.groupby("building_id"):
    df_b = df_b.sort_values("timestamp").reset_index(drop=True)

    # giữ bản sao để cập nhật Electricity bằng pred
    df_work = df_b.copy()

    # =============================
    # Loop theo horizon (recursive)
    # =============================
    for h in range(1, forecast_horizon + 1):
        model = models[h - 1]

        # Feature tại t
        X = df_work[X_cols]

        # Predict batch
        preds = model.predict(X)

        # Thời điểm dự đoán
        t0 = df_work["timestamp"]
        ts = t0 + timedelta(hours=h)

        # Actual (nếu có)
        df_actual = df_b[["timestamp", "Electricity"]] \
            .rename(columns={
                "timestamp": "ts",
                "Electricity": "actual"
            })

        df_h = pd.DataFrame({
            "building_id": building_id,
            "t0": t0,
            "horizon": h,
            "timestamp": ts,
            "pred": preds
        })

        df_h = df_h.merge(
            df_actual,
            left_on="timestamp",
            right_on="ts",
            how="left"
        ).drop(columns="ts")

        # =============================
        # Bỏ các dòng đã có
        # =============================
        mask_new = ~df_h.apply(
            lambda r: (r.t0, r.building_id, r.horizon) in existing_keys,
            axis=1
        )
        df_h = df_h.loc[mask_new]

        all_results.append(df_h)

        # =============================
        # UPDATE Electricity để recursive
        # =============================
        df_work["Electricity"] = preds

        # cập nhật lag & rolling cho bước sau
        df_work["lag_24"] = df_work["Electricity"].shift(24)
        df_work["rolling_24"] = df_work["Electricity"].rolling(24).mean()

# =============================
# 3. Gộp & lưu
# =============================
df_result = pd.concat(all_results, ignore_index=True)

df_result[[
    "building_id",
    "t0",
    "horizon",
    "timestamp",
    "actual",
    "pred"
]].to_csv(
    results_file,
    mode="a",
    index=False,
    header=not os.path.exists(results_file)
)

print(f"Done. Appended {len(df_result)} rows.")


Done. Appended 250965 rows.


In [10]:
df_result

,building_id,t0,horizon,timestamp,pred,actual
0,0,2017-08-31 00:00:00,1,2017-08-31 01:00:00,49.997841,23.00
1,0,2017-08-31 01:00:00,1,2017-08-31 02:00:00,29.613825,23.00
2,0,2017-08-31 02:00:00,1,2017-08-31 03:00:00,24.540413,23.00
3,0,2017-08-31 03:00:00,1,2017-08-31 04:00:00,22.834240,23.00
4,0,2017-08-31 04:00:00,1,2017-08-31 05:00:00,23.257473,10.00
...,...,...,...,...,...,...
250960,86,2017-12-31 19:00:00,1,2017-12-31 20:00:00,304.633820,203.75
250961,86,2017-12-31 20:00:00,1,2017-12-31 21:00:00,306.207336,210.74
250962,86,2017-12-31 21:00:00,1,2017-12-31 22:00:00,346.717407,666.04
250963,86,2017-12-31 22:00:00,1,2017-12-31 23:00:00,440.768982,725.08


In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# =============================
# Load kết quả
# =============================

df_result = pd.read_csv(results_file).dropna()
print(len(df_result))
# =============================
# Hàm tính SMAPE
# =============================
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# =============================
# Tính metric từng building
# =============================
metrics_by_building = []

for building, df_b in df_result.groupby("building_id"):
    actual = df_b["actual"].values
    pred = df_b["pred"].values
    
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    smape_val = smape(actual, pred)
    
    metrics_by_building.append({
        "building_id": building,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2,
        "SMAPE(%)": smape_val
    })

df_metrics = pd.DataFrame(metrics_by_building)
encoder = MultiColumnLabelEncoder.load(f"{model_dir}/categorical_encoder.pkl")
df_metrics["building_name"] = encoder.encoders["building_id"].inverse_transform(df_metrics["building_id"].astype(int))
df_metrics_sorted = df_metrics.sort_values("SMAPE(%)", ascending=True).reset_index(drop=True)
# =============================
# Tính trung bình trên tất cả building
# =============================
average_metrics = df_metrics_sorted[["RMSE", "MAE", "R2", "SMAPE(%)"]].mean()
average_metrics["building_id"] = "TBC"  # thêm dòng trung bình
df_metrics_sorted = pd.concat([df_metrics_sorted, pd.DataFrame([average_metrics])], ignore_index=True)

# =============================
# Hiển thị và lưu bảng metrics
# =============================
print(df_metrics_sorted)
df_metrics_sorted.to_csv(metric_file, index=False)


order = df_metrics_sorted[df_metrics_sorted["building_id"]!="TBC"]["building_id"]
df_result["building_id"] = pd.Categorical(df_result["building_id"], order, ordered=True)
df_result = df_result.sort_values(["building_id", "timestamp"])
df_result.to_csv(results_file)


248797
   building_id       RMSE        MAE         R2   SMAPE(%)  \
0           66  22.039427  14.512384   0.771203   2.866225   
1           60   3.913230   2.576268   0.219644   3.301658   
2           58  42.710907  33.760358   0.336185   3.618728   
3           44  34.701059  22.911365  -0.178857   3.634488   
4           37  49.861755  37.071978  -0.156219   3.814460   
..         ...        ...        ...        ...        ...   
83          81  16.183952  12.476889   0.002378  69.379779   
84          35  22.231588  21.551894 -12.851613  81.106690   
85           0  47.570566  36.854622   0.196392  93.800854   
86          38  50.802557  45.461243   0.465695  99.702020   
87         TBC  24.481026  18.595717   0.005730  17.802577   

             building_name  
0    Fox_education_Yolande  
1     Fox_education_Stacia  
2     Fox_education_Otilia  
3     Fox_education_Gloria  
4     Fox_education_Claude  
..                     ...  
83        Fox_office_Essie  
84     Fox_educa

In [12]:
df_metrics_sorted[:60]["building_name"].tolist()

['Fox_education_Yolande',
 'Fox_education_Stacia',
 'Fox_education_Otilia',
 'Fox_education_Gloria',
 'Fox_education_Claude',
 'Fox_office_Alice',
 'Fox_assembly_Renna',
 'Fox_education_Wendell',
 'Eagle_office_Francis',
 'Fox_education_Dominique',
 'Eagle_education_Will',
 'Fox_education_Eldon',
 'Fox_education_Leona',
 'Fox_public_Martin',
 'Eagle_education_Luther',
 'Fox_lodging_Wallace',
 'Fox_office_Bernard',
 'Fox_education_Marlana',
 'Fox_education_Leota',
 'Eagle_education_Samantha',
 'Fox_education_Virgil',
 'Fox_lodging_Frances',
 'Eagle_education_Shanna',
 'Fox_education_Andre',
 'Fox_education_Geoffrey',
 'Fox_education_Nilda',
 'Eagle_public_Preston',
 'Fox_education_Lilly',
 'Eagle_office_Sonya',
 'Fox_education_Jaclyn',
 'Fox_office_Edythe',
 'Fox_education_Long',
 'Fox_lodging_Isabell',
 'Fox_lodging_Stephen',
 'Fox_lodging_Morris',
 'Fox_education_Louie',
 'Fox_assembly_Johnnie',
 'Eagle_lodging_Stephanie',
 'Eagle_office_Randolph',
 'Fox_education_Ollie',
 'Eagle_offi